In [5]:
includet(joinpath(ENV["JULIA_DQMC"], "src/measure_framework.jl"))

In [139]:
measxml = "dqmc.meas.xml"
mp = measxml2MeasParams(measxml)

Loading DQMC parameters (.in.xml)
L = 4
SLICES = 100
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 100
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3


MeasParams
  requested: Array{Symbol}((1,))
  todo: Array{Symbol}((0,))
  meas_infile: String "dqmc.meas.xml"
  outfile: String "dqmc.meas.h5"
  p: Params
  dqmc_outfile: String "dqmc.out.h5"
  overwrite: Bool true
  num_threads: Int64 1
  walltimelimit: DateTime
  save_after: Int64 10
  to: TimerOutput
  confs_iterator_start: Int64 1


In [140]:
mp.requested

1-element Array{Symbol,1}:
 :chi_dyn

In [141]:
create_todolist!(mp)

1-element Array{Symbol,1}:
 :chi_dyn

In [147]:
println("\nLoading DQMC results...")

  greens = nothing; 
  mc = nothing;

  try
    global confs = ts_flat(mp.dqmc_outfile, "obs/configurations") # TODO: lazy load?
  catch err
    println("Couldn't read configuration data. Probably no configurations yet? Exiting.")
    exit()
  end

  if need_to_load_etgf(mp)
    if "greens" in listobs(mp.dqmc_outfile)
      greens = loadobs_frommemory(mp.dqmc_outfile, "obs/greens")
    else
      error("Couldn't find ETGF in $(mp.dqmc_outfile) although need_to_load_etgf == true.")
      # TODO: Calculate ETGF if it isn't found in dqmc output.
    end
  end

  # prepare dqmc framework if necessary
  if need_to_setup_mc(mp)
    mc = DQMC(mp.p)
    initialize_stack_for_measurements(mc, mp)
  end


  # ------------------- Create list of observables to measure --------------------
  num_confs = size(confs, ndims(confs))
  nsweeps = num_confs * mp.p.write_every_nth
  obs = NamedTuple() # list of observables to measure

  # chi_dyn
  if :chi_dyn_symm in mp.todo
    @addobs chi_dyn_symm Array{Float64, 3}
  end

  if :chi_dyn in mp.todo
    @addobs chi_dyn Array{Float64, 3}
  end
  
  # binder
  if :binder in mp.todo
    @addobs m2s Float64
    @addobs m4s Float64
    obs = add(obs, binder = create_or_load_obs("binder", Float64, name="binder"))
  end

  # etpc
  if :etpc_plus in mp.todo
    zero_etpc = zeros(Float64, mp.p.L, mp.p.L)
    @addlightobs etpc_plus zero_etpc
    @addlightobs etpc_minus zero_etpc
  end

  # zfpc
  if :zfpc_plus in mp.todo
    zero_zfpc = zeros(Float64, mp.p.L, mp.p.L)
    @addlightobs zfpc_plus zero_zfpc
    @addlightobs zfpc_minus zero_zfpc
  end

  # etcdc
  if :etcdc_plus in mp.todo
    zero_etcdc = zeros(ComplexF64, mp.p.L, mp.p.L)
    @addlightobs etcdc_plus zero_etcdc
    @addlightobs etcdc_minus zero_etcdc
  end

  # zfcdc
  if :zfcdc_plus in mp.todo
    zero_zfcdc = zeros(ComplexF64, mp.p.L, mp.p.L)
    @addlightobs zfcdc_plus zero_zfcdc
    @addlightobs zfcdc_minus zero_zfcdc
  end

  # sfdensity
  if :sfdensity in mp.todo
    @addlightobs sfdensity Float64
  end

  # # tdgfs
  # if mp.tdgfs
  #   # create zero element for LightObservable
  #   Nflv = mc.p.flv * mc.l.sites
  #   tdgf_size = (Nflv, Nflv, mc.p.slices)
  #   zero_tdgf = zeros(geltype(mc), tdgf_size)
  #   (obs = add(obs, tdgfs_Gt0 = LightObservable(zero_tdgf, name="Time-displaced Green's function G(tau,0) (TDGF Gt0)", alloc=num_confs)))
  #   (obs = add(obs, tdgfs_G0t = LightObservable(zero_tdgf, name="Time-displaced Green's function G(0, tau) (TDGF G0t)", alloc=num_confs)))
  # end



  # --------------------- Measure -------------------------
  print("\nPrepared Observables ");
  println(keys(obs))
  println()
  reset_timer!(mp.to)


Loading DQMC results...

Prepared Observables (:chi_dyn,)



 ──────────────────────────────────────────────────────────────────
                           Time                   Allocations      
                   ──────────────────────   ───────────────────────
 Tot / % measured:     11.3ms / 0.00%           12.3KiB / 0.00%    

 Section   ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────
 ──────────────────────────────────────────────────────────────────

In [148]:
  measure(mp, obs, confs, greens, mc)

Measuring ...
1
2
3
4
5


 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            2.66s / 0.54%           16.7MiB / 5.29%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 measure loop               1   14.2ms   100%  14.2ms    905KiB  100%    905KiB
   bosonic                  5   1.11ms  7.79%   222μs    846KiB  93.5%   169KiB
     chi                    5   1.08ms  7.60%   216μs    844KiB  93.3%   169KiB
     binder                 5    799ns  0.01%   160ns     0.00B  0.00%    0.00B
   fermionic                5   17.9μs  0.13%  3.58μs   6.34KiB  0.70%  1.27KiB
     calculate TDGFs        5   1.20μs  0.01%   240ns     0.00B  0.00%    0.00B
     sfdensity              5    600ns 

In [149]:
  export_results(mp, obs, nsweeps)

Calculating errors and exporting...
Done. Exported to dqmc.meas.h5.
